<a href="https://colab.research.google.com/github/ucaokylong/LLM_learning/blob/main/function_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAI Function Calling 101
Một trong những khó khăn khi sử dụng các mô hình ngôn ngữ lớn (LLM) như ChatGPT là chúng không tạo ra đầu ra dữ liệu có cấu trúc. Điều này rất quan trọng đối với các hệ thống lập trình phụ thuộc phần lớn vào dữ liệu có cấu trúc để tương tác. Ví dụ, nếu bạn muốn xây dựng một chương trình phân tích cảm xúc của một bài đánh giá phim, bạn có thể phải thực hiện một đoạn code trông giống như sau:

```
prompt = f'''
Please perform a sentiment analysis on the following movie review:
{MOVIE_REVIEW_TEXT}
Please output your response as a single word: either "Positive" or "Negative". Do not add any extra characters.
'''
```

Vấn đề là điều này không phải lúc nào cũng hiệu quả. Khá phổ biến là LLM sẽ thêm vào một dấu chấm không mong muốn hoặc giải thích dài hơn như: "Cảm xúc của bộ phim này là: Tích cực." Mặc dù bạn có thể sử dụng biểu thức chính quy (regex) để lấy ra câu trả lời (🤢), nhưng rõ ràng đây không phải là lý tưởng. Điều lý tưởng sẽ là nếu LLM trả về kết quả dưới dạng cấu trúc JSON như sau:

```
{
    'sentiment': 'positive'
}
```

OpenAI đã giới thiệu một tính năng mới gọi là function calling, giúp giải quyết vấn đề trên. Function calling chính là câu trả lời cho vấn đề trên. Jupyter notebook này sẽ minh họa một ví dụ đơn giản về cách sử dụng function calling mới của OpenAI trong Python. Nếu bạn muốn xem tài liệu đầy đủ, [vui lòng xem liên kết này](https://platform.openai.com/docs/guides/gpt/function-calling).

## Notebook Setup
Hãy bắt đầu với các import. Bây giờ, có thể bạn đã cài đặt client Python `openai`, nhưng rất có thể bạn cần nâng cấp nó để có được chức năng function calling mới. Đây là cách nâng cấp trong Terminal / Powershell của bạn bằng `pip`:

```
pip install openai --upgrade
```

In [ ]:
!pip install openai --upgrade

In [ ]:
# Importing the necessary Python libraries
from IPython.display import clear_output
import os
import json
import yaml
import openai

Để kiểm tra chức năng function calling, tôi đã viết một đoạn "About Me" ngắn chứa các sự thật cụ thể mà chúng ta sẽ phân tích thành các cấu trúc dữ liệu phù hợp, bao gồm số nguyên và chuỗi. Hãy tải văn bản này vào

In [ ]:
from google.colab import userdata


# Loading the "About Me" text from local file
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
about_me = "Hello! My name is David Hundley. I am a principal machine learning engineer at State Farm. I enjoy learning about AI and teaching what I learn back to others. I have two daughters. I drive a Tesla Model 3, and my favorite video game series is The Legend of Zelda."

print(about_me)


## The Pre-Function Calling Days
Trước khi chúng ta minh họa function calling, hãy minh họa cách chúng ta đã sử dụng prompt engineering và Regex để tạo ra một JSON có cấu trúc mà chúng ta có thể làm việc một cách lập trình sau này.

In [ ]:
# Engineering a prompt to extract as much information from "About Me" as a JSON object
about_me_prompt = f'''
Please extract information as a JSON object. Please look for the following pieces of information.
Name
Job title
Company
Number of children as a single number
Car make
Car model
Favorite video game series

This is the body of text to extract the information from:
{about_me}
'''

In [ ]:
# Getting the response back from ChatGPT (gpt-3.5-turbo)
openai_response = openai.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [{'role': 'user', 'content': about_me_prompt}]
)

In [ ]:
# Loading the response as a JSON object
json_response = json.loads(openai_response.choices[0].message.content)
print(json_response)

{'Name': 'David Hundley', 'Job title': 'Principal Machine Learning Engineer', 'Company': 'State Farm', 'Number of children': 2, 'Car make': 'Tesla', 'Car model': 'Model 3', 'Favorite video game series': 'The Legend of Zelda'}


## Using the New Function Calling Capabilities
Bây giờ chúng ta đã minh họa cách chúng ta đã từng lấy được JSON có cấu trúc trong những ngày trước khi có function calling, hãy chuyển sang cách chúng ta có thể sử dụng function calling để trích xuất các kết quả tương tự nhưng theo cách nhất quán hơn cho việc sử dụng hệ thống của chúng ta. Chúng ta sẽ bắt đầu đơn giản hơn với một hàm tùy chỉnh duy nhất và sau đó giải quyết một số chức năng "nâng cao" hơn.

In [ ]:
# Defining our initial extract_person_info function
def extract_person_info(name, job_title, num_children):
    '''
    Prints basic "About Me" information

    Inputs:
        name (str): Name of the person
        job_title (str): Job title of the person
        num_chilren (int): The number of children the parent has.
    '''

    print(f'This person\'s name is {name}. Their job title is {job_title}, and they have {num_children} children.')

In [ ]:
# Defining how we want ChatGPT to call our custom functions
my_custom_functions = [
    {
        'name': 'extract_person_info',
        'description': 'Get "About Me" information from the body of the input text',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the person'
                },
                'job_title': {
                    'type': 'string',
                    'description': 'Job title of the person'
                },
                'num_children': {
                    'type': 'integer',
                    'description': 'Number of children the person is a parent to'
                }
            }
        }
    }
]

In [ ]:
# Getting the response back from ChatGPT (gpt-3.5-turbo)
openai_response = openai.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [{'role': 'user', 'content': about_me}],
    functions = my_custom_functions,
    function_call = 'auto'
)

print(openai_response)

ChatCompletion(id='chatcmpl-94o26OACU2zjAMUwNvIDsXvqnRCQn', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"name":"David Hundley","job_title":"Principal Machine Learning Engineer","num_children":2}', name='extract_person_info'), tool_calls=None))], created=1710932666, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_4f0b692a78', usage=CompletionUsage(completion_tokens=30, prompt_tokens=147, total_tokens=177))


In [ ]:
print(openai_response.choices[0].message.function_call)

FunctionCall(arguments='{"name":"David Hundley","job_title":"Principal Machine Learning Engineer","num_children":2}', name='extract_person_info')


### What if the prompt I submit doesn't contain the information I want to extract per my custom function?
Copy code
Trong ví dụ ban đầu của chúng ta, hàm tùy chỉnh đã tìm cách trích xuất ba thông tin rất cụ thể và chúng ta đã chứng minh rằng điều này hoạt động thành công bằng cách truyền văn bản "About Me" tùy chỉnh của tôi dưới dạng một prompt. Nhưng bạn có thể tự hỏi, điều gì sẽ xảy ra nếu bạn truyền vào bất kỳ prompt nào khác không chứa thông tin đó?

Hãy nhớ lại rằng chúng ta đã đặt một tham số trong lệnh gọi API client của mình gọi là function_call mà chúng ta đặt thành auto. Chúng ta sẽ khám phá điều này sâu hơn nữa trong phần tiếp theo, nhưng về cơ bản, tham số này đang yêu cầu ChatGPT sử dụng phán đoán tốt nhất của nó để tìm ra khi nào cần cấu trúc đầu ra cho một trong các hàm tùy chỉnh của chúng ta.

Vậy điều gì sẽ xảy ra khi chúng ta gửi một prompt không khớp với bất kỳ hàm tùy chỉnh nào của chúng ta? Nói một cách đơn giản, nó sẽ mặc định trở lại hành vi thông thường như thể function calling không tồn tại. Hãy kiểm tra điều này với một prompt tùy ý: "Tháp Eiffel cao bao nhiêu?"

In [ ]:
# Getting the response back from ChatGPT (gpt-3.5-turbo)
openai_response = openai.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [{'role': 'user', 'content': 'How tall is the Eiffel Tower?'}],
    functions = my_custom_functions,
    function_call = 'auto'
)

print(openai_response)

ChatCompletion(id='chatcmpl-94o7fPVQErjmsLTu7ynKXW3BQNxdG', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The Eiffel Tower is 1,063 feet (324 meters) tall, including antennas.', role='assistant', function_call=None, tool_calls=None))], created=1710933011, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_4f0b692a78', usage=CompletionUsage(completion_tokens=21, prompt_tokens=97, total_tokens=118))


In [ ]:
# Defining a function to extract only vehicle information
def extract_vehicle_info(vehicle_make, vehicle_model):
    '''
    Prints basic vehicle information

    Inputs:
        - vehicle_make (str): Make of the vehicle
        - vehicle_model (str): Model of the vehicle
    '''

    print(f'Vehicle make: {vehicle_make}\nVehicle model: {vehicle_model}')



# Defining a function to extract all information provided in the original "About Me" prompt
def extract_all_info(name, job_title, num_children, vehicle_make, vehicle_model, company_name, favorite_vg_series):
    '''
    Prints the full "About Me" information

    Inputs:
        - name (str): Name of the person
        - job_title (str): Job title of the person
        - num_chilren (int): The number of children the parent has
        - vehicle_make (str): Make of the vehicle
        - vehicle_model (str): Model of the vehicle
        - company_name (str): Name of the company the person works for
        - favorite_vg_series (str): Person's favorite video game series.
    '''

    print(f'''
    This person\'s name is {name}. Their job title is {job_title}, and they have {num_children} children.
    They drive a {vehicle_make} {vehicle_model}.
    They work for {company_name}.
    Their favorite video game series is {favorite_vg_series}.
    ''')

In [ ]:
# Defining how we want ChatGPT to call our custom functions
my_custom_functions = [
    {
        'name': 'extract_person_info',
        'description': 'Get "About Me" information from the body of the input text',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the person'
                },
                'job_title': {
                    'type': 'string',
                    'description': 'Job title of the person'
                },
                'num_children': {
                    'type': 'integer',
                    'description': 'Number of children the person is a parent to'
                }
            }
        }
    },
    {
        'name': 'extract_vehicle_info',
        'description': 'Extract the make and model of the person\'s car',
        'parameters': {
            'type': 'object',
            'properties': {
                'vehicle_make': {
                    'type': 'string',
                    'description': 'Make of the person\'s vehicle'
                },
                'vehicle_model': {
                    'type': 'string',
                    'description': 'Model of the person\'s vehicle'
                }
            }
        }
    },
    {
        'name': 'extract_all_info',
        'description': 'Extract all information about a person including their vehicle make and model',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the person'
                },
                'job_title': {
                    'type': 'string',
                    'description': 'Job title of the person'
                },
                'num_children': {
                    'type': 'integer',
                    'description': 'Number of children the person is a parent to'
                },
                'vehicle_make': {
                    'type': 'string',
                    'description': 'Make of the person\'s vehicle'
                },
                'vehicle_model': {
                    'type': 'string',
                    'description': 'Model of the person\'s vehicle'
                },
                'company_name': {
                    'type': 'string',
                    'description': 'Name of the company the person works for'
                },
                'favorite_vg_series': {
                    'type': 'string',
                    'description': 'Name of the person\'s favorite video game series'
                }
            }
        }
    }
]

Now let's demonstrate what happens when we apply 3 different samples to all of the custom functions.

In [ ]:
# Defining a list of samples
samples = [
    about_me,
    'My name is David Hundley. I am a principal machine learning engineer, and I have two daughters.',
    'She drives a Kia Sportage.'
]

In [ ]:
# Iterating over the three samples
for i, sample in enumerate(samples):

    print(f'Sample #{i + 1}\'s results:')

    # Getting the response back from ChatGPT (gpt-3.5-turbo)
    openai_response = openai.chat.completions.create(
        model = 'gpt-3.5-turbo',
        messages = [{'role': 'user', 'content': sample}],
        functions = my_custom_functions,
        function_call = 'auto'
    )

    # Printing the sample's response
    print(openai_response)

Sample #1's results:
ChatCompletion(id='chatcmpl-94o8LMn34OQ4Nw6nG7Aq12ehlMhjG', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"name":"David Hundley","job_title":"Principal Machine Learning Engineer","num_children":2,"vehicle_make":"Tesla","vehicle_model":"Model 3","company_name":"State Farm","favorite_vg_series":"The Legend of Zelda"}', name='extract_all_info'), tool_calls=None))], created=1710933053, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_4f0b692a78', usage=CompletionUsage(completion_tokens=58, prompt_tokens=320, total_tokens=378))
Sample #2's results:
ChatCompletion(id='chatcmpl-94o8NGEFk16kB8YZvuJpk3c5cl5JN', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"name":"David Hundley","job_title":"Princip

With each of the respective prompts, ChatGPT selected the correct custom function, and we can specifically note that in the `name` value under `function_call` in the API's response object. In addition to this being a handy way to identify which function to use the arguments for, we can programmatically map our actual custom Python function to this value to run the correct code appropriately.

In [ ]:
# Iterating over the three samples
for i, sample in enumerate(samples):

    print(f'Sample #{i + 1}\'s results:')

    # Getting the response back from ChatGPT (gpt-3.5-turbo)
    openai_response = openai.chat.completions.create(
        model = 'gpt-3.5-turbo',
        messages = [{'role': 'user', 'content': sample}],
        functions = my_custom_functions,
        function_call = 'auto'
    ).choices[0].message

    # Checking to see that a function call was invoked
    if openai_response.function_call:
        # Checking to see which specific function call was invoked
        function_called = openai_response.function_call.name

        # Extracting the arguments of the function call
        function_args = json.loads(openai_response.function_call.arguments)

        # Invoking the proper functions
        if function_called == 'extract_person_info':
            extract_person_info(*list(function_args.values()))
        elif function_called == 'extract_vehicle_info':
            extract_vehicle_info(*list(function_args.values()))
        elif function_called == 'extract_all_info':
            extract_all_info(*list(function_args.values()))

Sample #1's results:

    This person's name is David Hundley. Their job title is Principal Machine Learning Engineer, and they have 2 children.
    They drive a Tesla Model 3.
    They work for State Farm.
    Their favorite video game series is The Legend of Zelda.
    
Sample #2's results:
This person's name is David Hundley. Their job title is Principal Machine Learning Engineer, and they have 2 children.
Sample #3's results:
Vehicle make: Kia
Vehicle model: Sportage


## OpenAI Function Calling with LangChain
Given its popularity amongst the Generative AI community, I thought I'd re-visit this notebook and add some code to show how you might make use of this exact same functionality in LangChain

In [ ]:
!pip install langchain langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [ ]:
# Importing the LangChain objects
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts.chat import ChatPromptTemplate
from langchain.chains.openai_functions import create_structured_output_chain

In [ ]:
# Setting the proper instance of the OpenAI model
llm = ChatOpenAI(model = 'gpt-3.5-turbo-0613')

# Setting a LangChain ChatPromptTemplate
chat_prompt_template = ChatPromptTemplate.from_template('{my_prompt}')

# Setting the JSON schema for extracting vehicle information
langchain_json_schema = {
    'name': 'extract_vehicle_info',
    'description': 'Extract the make and model of the person\'s car',
    'type': 'object',
    'properties': {
        'vehicle_make': {
            'title': 'Vehicle Make',
            'type': 'string',
            'description': 'Make of the person\'s vehicle'
        },
        'vehicle_model': {
            'title': 'Vehicle Model',
            'type': 'string',
            'description': 'Model of the person\'s vehicle'
        }
    }
}

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
# Defining the LangChain chain object for function calling
chain = create_structured_output_chain(output_schema = langchain_json_schema,
                                       llm = llm,
                                       prompt = chat_prompt_template)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `create_structured_output_chain` was deprecated in LangChain 0.1.1 and will be removed in 0.2.0. Use create_structured_output_runnable instead.
  warn_deprecated(


In [ ]:
# Getting results with a demo prompt
print(chain.invoke(input = {'my_prompt': 'I drive a Tesla Model 3'}))

{'my_prompt': 'I drive a Tesla Model 3', 'function': {'vehicle_make': 'Tesla', 'vehicle_model': 'Model 3'}}


In [ ]:
clear_output()